# Producing the Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

## Notebook Initialization

In [2]:
%autoreload
import numpy as np
import pickle
from tqdm import tqdm_notebook as tqdm

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from darkhistory.spec.spectrum import Spectrum
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
import darkhistory.physics as phys
import darkhistory.utilities as utils

from scipy.interpolate import interp1d
from scipy.interpolate import RegularGridInterpolator

# np.set_printoptions(threshold=np.nan)

/usr/local/lib/python3.7/site-packages/matplotlib/__init__.py:535: UserWarning: matplotlibrc text.usetex option can not be used unless TeX is installed on your system
  warnings.warn('matplotlibrc text.usetex option can not be used unless '
/usr/local/lib/python3.7/site-packages/matplotlib/__init__.py:543: UserWarning: matplotlibrc text.usetex can not be used with *Agg backend unless dvipng-1.6 or later is installed on your system
  'your system' % dvipng_req)


## Import Raw Data

In [5]:
standard_tfs = False

if standard_tfs == True:
    direc = '/Users/gregoryridgway/Desktop/DM/tfs_standard_xe_xHe_full_rs/'
    xes_str = ['0.000E+00']
    extra= '_full_rs'
else:
    direc = '/Users/gregoryridgway/Desktop/DM/tfs_tmp/'
    xes_str = [
        '4.540E-05', '2.746E-04', '1.659E-03', '9.952E-03', 
        '2.660E-02', '5.732E-02', '1.192E-01', '1.680E-01', '2.142E-01', '2.689E-01', '3.823E-01', 
        '5.000E-01', '5.987E-01', '6.900E-01', '8.022E-01', 
        '9.309E-01', '9.879E-01', '9.980E-01', '9.997E-01'   
    ]
    extra='_full_rs'

file_names = [
    direc
    +'transferfunction_withloweng_cmbloss_60eV_fixed_binning_complete_xe_'
    +string+extra+'_nointerp.p'
    for string in xes_str    
]

raw_tf_tuple = [pickle.load(open(file_name, "rb")) for file_name in file_names]

# xe, in_eng, rs, eng, type={highengphot, lowengphot, lowengelec}
raw_tf_list = np.array([tup[0] for tup in raw_tf_tuple])
# Truncate the energy binning, because in_eng does not go up to the end.
# raw_tf_list = raw_tf_list[:,:,:,:494,:]
# xe, in_eng, rs
CMB_engloss_list = np.array([tup[1] for tup in raw_tf_tuple])
# xe, in_eng, rs, {H-ion, Exc, Heating, Cont.}
highdeposited_list = np.array([tup[2] for tup in raw_tf_tuple])
# xe, in_eng, rs, type={mwimp, rs}
info_list = np.array([tup[3] for tup in raw_tf_tuple])
# xe, in_eng, rs, type={photeng, eleceng}, eng
abscissae_list = np.array([tup[4] for tup in raw_tf_tuple])
# abscissae_list = abscissae_list[:,:,:,:,:494]

In [6]:
# This checks that all of the abscissae are equal.
for i in np.arange(info_list.shape[0]):
    for j in np.arange(info_list.shape[1]):
        if not np.array_equal(
            info_list[i,j,:,1],
            info_list[0,0,:,1]
        ):
            print('NOPE!')
        for k in np.arange(info_list.shape[2]):
            if not np.array_equal(
                info_list[i,:,k,0],
                info_list[0,:,0,0]
            ):
                print('NOPE!')
            if not np.array_equal(
                abscissae_list[i,j,k,0,:],
                abscissae_list[0,0,0,0,:]
            ):
                print('NOPE!')
            if not np.array_equal(
                abscissae_list[i,j,k,1,:],
                abscissae_list[0,0,0,1,:]
            ):
                print('NOPE!')
                
if standard_tfs:
    xe_arr = np.array([0])
else:
    xe_arr = 0.5 + 0.5*np.tanh(
        [-5., -4.1, -3.2, -2.3, 
         -1.8, -1.4, -1, -0.8, -0.65, -0.5, -0.24, 
         0.0, 0.2, 0.4, 0.7, 
         1.3, 2.2, 3.1, 4.]
    )

rs_arr    = info_list[0,0,:,1]
mwimp_arr = info_list[0,:,0,0]
photeng = abscissae_list[0,0,0,0,:]
eleceng = abscissae_list[0,0,0,1,:]

## Raw Data Manipulation

The transfer function is calculated by injecting 2 photons at the input energy abscissa. However, the first step is to assign $x$ photons to the top bin of the *output* photon abscissa, such that $x$ times the energy of the top bin is 2 times the injection energy. Because of the misalignment between the two abscissae, as well as the fact that we want to transfer function for a single electron, we have to normalize the results first.

First, we construct two arrays: a list of output abscissae (for both photons and electrons) given the injection abscissa, and a list of the energy of the *output* photon energy bin where the injected photons are assigned to. Then we compute the normalization factor. 

In [7]:
# This treatment is consistent with ih_transferfunction
photeng_bin_width = np.diff(np.log(spectools.get_bin_bound(photeng))) * photeng
eleceng_bin_width = np.diff(np.log(spectools.get_bin_bound(eleceng))) * eleceng


# Original injection energies
top_photeng_bins = np.array(
    [
        photeng[photeng <= mwimp][-1]
        for mwimp in mwimp_arr
    ]
)

norm_fac_arr = 2*mwimp_arr/top_photeng_bins

Dividing by the normalization factor gives the spectra for the injection of 1 photon, with energy given by `top_photeng_bins`. 

Now we construct a raw array of all of the transfer functions, on which we will perform an interpolation to extract a set of transfer functions for use, given a chosen abscissa.

In [8]:
# Multiplying by bin width converts all the eleceng array into number of particles. 
# Indexing: xe, in_eng, rs, eng, type={highengphot, lowengphot, lowengelec}

phot_tf_raw = np.array([
    [
        [
            raw_tf[i,j,:,0]/norm_fac*photeng_bin_width
                for (j, rs) in enumerate(rs_arr)   
        ]
        for (i, (in_eng, norm_fac)) in enumerate(zip(mwimp_arr, norm_fac_arr))
        
    ] for raw_tf in tqdm(raw_tf_list)
])

lowengphot_tf_raw = np.array([
    [
        [
            raw_tf[i,j,:,1]/norm_fac*photeng_bin_width 
                for (j, rs) in enumerate(rs_arr)   
        ]
        for (i, (in_eng, norm_fac)) in enumerate(zip(mwimp_arr, norm_fac_arr))
        
    ] for raw_tf in tqdm(raw_tf_list)
])

lowengelec_tf_raw = np.array([
    [
        [
            raw_tf[i,j,:,2]/norm_fac*eleceng_bin_width 
                for (j, rs) in enumerate(rs_arr)   
        ]
        for (i, (in_eng, norm_fac)) in enumerate(zip(mwimp_arr, norm_fac_arr))
        
    ] for raw_tf in tqdm(raw_tf_list)
])

# xe, in_eng, rs
CMB_engloss = CMB_engloss_list/np.outer(norm_fac_arr, np.ones_like(rs_arr))

# xe, in_eng, rs, {H-Ion, Exc, Heat, Cont}
denom_highdeposited = np.ones((xe_arr.size, rs_arr.size, 4, norm_fac_arr.size))
denom_highdeposited *= norm_fac_arr
denom_highdeposited = np.rollaxis(denom_highdeposited, 3, 1)
highdeposited = highdeposited_list/denom_highdeposited

In [9]:
photeng_high = photeng[photeng > 60]
print(photeng_high.shape)

new_phot_tf_raw = phot_tf_raw
new_lowengphot_tf_raw = lowengphot_tf_raw
new_lowengelec_tf_raw = lowengelec_tf_raw
new_cmbloss_raw = CMB_engloss
new_highdeposited_raw = highdeposited

(327,)


Now, we construct the list of `Spectrum` objects from our raw array of transfer functions...

In [10]:
# spec_type = 'N' for eleceng, because we have already multiplied in the bin width earlier. 

#xe, in_eng, rs, eng
photspec_list = [
    [
        [
            Spectrum(
                photeng, phot_tf[i,j,:], spec_type='N', 
                rs = rs, in_eng = in_eng
            ) for (j, rs) in enumerate(rs_arr)
        ] for (i, in_eng) in enumerate(photeng_high)
    ] for phot_tf in tqdm(new_phot_tf_raw)
]

lowengphotspec_list = [
    [
        [
            Spectrum(
                photeng, lowengphot_tf[i,j,:], spec_type='N', 
                rs = rs, in_eng = in_eng
            ) for (j, rs) in enumerate(rs_arr)
        ] for (i, in_eng) in enumerate(photeng_high)
    ] for lowengphot_tf in tqdm(new_lowengphot_tf_raw)
]

lowengelecspec_list = [
    [
        [
            Spectrum(
                eleceng, lowengelec_tf[i,j,:], spec_type='N', 
                rs = rs, in_eng = in_eng
            ) for (j, rs) in enumerate(rs_arr)
        ] for (i, in_eng) in enumerate(photeng_high)
    ] for lowengelec_tf in tqdm(new_lowengelec_tf_raw)
]


Finally, we can construct the `TransferFuncList` from this.

In [11]:
tfunclist_photspec_list = [
    tflist.TransferFuncList(
        [
            tf.TransFuncAtEnergy(spec_arr, dlnz=0.001)
            for spec_arr in photspec_xe
        ]
    ) for photspec_xe in tqdm(photspec_list)
]

tfunclist_lowengphotspec_list = [
    tflist.TransferFuncList(
        [
            tf.TransFuncAtEnergy(spec_arr, dlnz=0.001)
            for spec_arr in lowengphotspec_xe
        ]
    ) for lowengphotspec_xe in tqdm(lowengphotspec_list)
]
    

tfunclist_lowengelecspec_list = [
    tflist.TransferFuncList(
        [
            tf.TransFuncAtEnergy(spec_arr, dlnz=0.001)
            for spec_arr in lowengelecspec_xe
        ]
    ) for lowengelecspec_xe in tqdm(lowengelecspec_list)
]

We will now transpose all of the `TransferFuncList` objects so that they are now all of type `'rs'`.

In [12]:
for tflist_hep, tflist_lep, tflist_lee in zip(
    tfunclist_photspec_list, tfunclist_lowengphotspec_list, tfunclist_lowengelecspec_list):
    tflist_hep.transpose()
    tflist_lep.transpose()
    tflist_lee.transpose()
    for i, tf_hep, tf_lep, tf_lee in zip(np.arange(327), tflist_hep, tflist_lep, tflist_lee):
        if np.any(np.isnan(tf_hep._grid_vals)):
            print(np.where(np.isnan(tf_hep._grid_vals)),'here!')
        if np.any(np.isnan(tf_lep._grid_vals)):
            print(np.where(np.isnan(tf_lep._grid_vals)),'here1!')
        if np.any(np.isnan(tf_lee._grid_vals)):
            print(np.where(np.isnan(tf_lee._grid_vals)),'here2!')
        
        tf_hep._grid_vals[np.where(tf_hep.grid_vals < 0)] = 1e-100
        tf_lep._grid_vals[np.where(tf_lep.grid_vals < 0)] = 1e-100
        tf_lee._grid_vals[np.where(tf_lee.grid_vals < 0)] = 1e-100

#low energy photon spectrum with the CMB subtracted
tfunclist_lowengphotspec_subtracted_list = tfunclist_lowengphotspec_list.copy()

And, finally, save the transfer functions.

In [13]:
def save_as_pickled_object(obj, filepath):
    """
    This is a defensive way to write pickle.write, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    bytes_out = pickle.dumps(obj)
    n_bytes = sys.getsizeof(bytes_out)
    with open(filepath, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])

In [14]:
if standard_tfs:
    direc = "/Users/gregoryridgway/Desktop/DM/tfs_standard_xe_xHe_full_rs/"
    string="_standard_xe_xHe_full_rs"
else:
    direc = "/Users/gregoryridgway/Desktop/DM/tfs_tmp/"
    string = "_xHe0"

save_as_pickled_object(tfunclist_photspec_list, direc+"tfunclist_photspec_60eV_complete"+string+".raw")
save_as_pickled_object(tfunclist_lowengphotspec_list, direc+"tfunclist_lowengphotspec_60eV_complete"+string+".raw")
save_as_pickled_object(tfunclist_lowengelecspec_list, direc+"tfunclist_lowengelecspec_60eV_complete"+string+".raw")
save_as_pickled_object(new_cmbloss_raw, direc+"CMB_engloss_60eV_complete"+string+".raw")
save_as_pickled_object(new_highdeposited_raw, direc+"highdeposited_60eV_complete"+string+".raw")